In [ ]:
import xarray as xr
import numpy as np

In [ ]:
ds = xr.open_dataset('../data/ecefiles/runoff_maps.nc')
lon = ds.lon.values
lat = ds.lat.values
lons,lats = np.meshgrid(lon,lat)

In [ ]:
#Get distance from Antarctic coast
n = 45

mask = np.where(ds['drainage_basin_id']>-1,1,1e12)[:n,:]
cdist = np.zeros((len(lat),len(lon)))+1000
Re = 6371 #km

for j,jj in enumerate(lat[:n+10]):
    for i,ii in enumerate(lon):
        #Haversine formula
        dist = 2*Re*np.arcsin(np.sqrt(np.sin(np.deg2rad(0.5*(lats[:n,:]-jj)))**2+np.cos(np.deg2rad(jj))*np.cos(np.deg2rad(lats[:n,:]))*np.sin(np.deg2rad(0.5*(lons[:n,:]-ii)))**2))
        #dist = (lons-ii)**2+(lats-jj)**2
        if j < n:
            dist[j,i] = 1e12
        cdist[j,i] = np.minimum(1000,np.min(dist*mask))

In [ ]:
#Create some reasonable catchment mask (not used)

drain = ds['drainage_basin_id']

#Separate Ross vs Wedd
drain = np.where(np.logical_and(np.logical_and(drain==66,lons>90),lons<270),67,drain)

#Split EAIS
drain = np.where(np.logical_and(np.logical_and(drain>65,lons>350),lats>-77),68,drain)
drain = np.where(np.logical_and(np.logical_and(drain>65,lons<180),lats>-77),68,drain)

#Split Amun
drain = np.where(np.logical_and(np.logical_and(np.logical_and(drain>65,lons>210),lons<280),lats>-77),69,drain)

#Split Pens
drain = np.where(np.logical_and(np.logical_and(np.logical_and(drain==66,lons>292),lons<310),lats>-70),70,drain)
drain = np.where(np.logical_and(np.logical_and(np.logical_and(drain==66,lons>292),lons<295),lats>-74),70,drain)
drain = np.where(np.logical_and(np.logical_and(np.logical_and(np.logical_and(drain==66,lons>280),lons<293),lats<-72),lats>-74),70,drain)

#Add to Amun
drain = np.where(np.logical_and(np.logical_and(np.logical_and(drain==66,lons>280),lons<295),lats>-72),69,drain)

In [ ]:
#Create calving mask

calv = ds['calving_point_id']

#Remove old mask
calv = np.where(calv==66,-2,calv)

#Amun
calv = np.where(np.logical_and(np.logical_and(np.logical_and(drain<0,lons>210),lons<270),lats<-72),69,calv)

#Ross
calv = np.where(np.logical_and(np.logical_and(np.logical_and(drain<0,lons>150),lons<210),lats<-72),67,calv)
calv = np.where(np.logical_and(np.logical_and(np.logical_and(drain<0,lons>120),lons<180),lats<-69),67,calv)
calv = np.where(np.logical_and(np.logical_and(np.logical_and(drain<0,lons>120),lons<170),lats<-67),67,calv)

#EAIS
calv = np.where(np.logical_and(np.logical_and(np.logical_and(drain<0,lons<150),lons>42),lats<-64),68,calv)
calv = np.where(np.logical_and(np.logical_and(np.logical_and(drain<0,lons>10),lons<42),lats<-66),68,calv)
calv = np.where(np.logical_and(np.logical_and(drain<0,lons<42),lats<-67),68,calv)
calv = np.where(np.logical_and(np.logical_and(np.logical_and(drain<0,lons>330),lats>-72),lats<-68),68,calv)

#Weddell
calv = np.where(np.logical_and(np.logical_and(np.logical_and(np.logical_and(drain<0,calv<0),lons>295),lats<-68),lons<350),66,calv)
calv = np.where(np.logical_and(np.logical_and(np.logical_and(np.logical_and(drain<0,calv<0),lons>310),lats<-58),lons<318),66,calv)
calv = np.where(np.logical_and(np.logical_and(np.logical_and(np.logical_and(drain<0,calv<0),lons>305),lats<-65),lons<318),66,calv)

#Pens
calv = np.where(np.logical_and(np.logical_and(np.logical_and(np.logical_and(drain<0,calv<0),lons>298),lats<-58),lons<318),70,calv)
calv = np.where(np.logical_and(np.logical_and(np.logical_and(np.logical_and(drain<0,calv<0),lons>300),lats<-62),lons<318),70,calv)

calv = np.where(np.logical_and(calv==67,cdist>300),-2,calv)
calv = np.where(np.logical_and(calv==68,cdist>300),-2,calv)
calv = np.where(np.logical_and(calv==69,cdist>300),-2,calv)
calv = np.where(np.logical_and(lats<-68,np.logical_and(calv==66,cdist>400)),-2,calv)
calv = np.where(np.logical_and(lats<-66,np.logical_and(calv==66,cdist>600)),-2,calv)

#Retain -2 over land
#calv = np.where(ds['calving_point_id']==-1,-1,calv)

In [ ]:
#Create runoff/basal melt mask

arr = ds['arrival_point_id']

#Remove old mask
arr = np.where(arr==66,-2,arr)

#Amun
arr = np.where(np.logical_and(np.logical_and(np.logical_and(drain<0,lons>225),lons<270),lats<-72),69,arr)
arr = np.where(np.logical_and(np.logical_and(np.logical_and(drain<0,lons>252),lons<275),lats<-71),69,arr)

#Ross
arr = np.where(np.logical_and(np.logical_and(np.logical_and(drain<0,lons>150),lons<180),lats<-76),67,arr)

#EAIS
arr = np.where(np.logical_and(np.logical_and(np.logical_and(drain<0,lons<140),lons>80),lats<-64.5),68,arr)
arr = np.where(np.logical_and(np.logical_and(np.logical_and(drain<0,lons<80),lons>75),lats<-66),68,arr)
arr = np.where(np.logical_and(np.logical_and(np.logical_and(drain<0,lons<75),lons>70),lats<-67),68,arr)
arr = np.where(np.logical_and(np.logical_and(np.logical_and(drain<0,lons<150),lons>140),lats<-65),68,arr)

#Wedd
arr = np.where(np.logical_and(np.logical_and(np.logical_and(drain<0,lons>295),lats<-75.5),lons<350),66,arr)
arr = np.where(np.logical_and(np.logical_and(np.logical_and(drain<0,lons>295),lats<-74.5),lons<310),66,arr)

#Pens
arr = np.where(np.logical_and(np.logical_and(np.logical_and(np.logical_and(drain<0,lons>298),lats<-64.5),lons<304),lats>-70),70,arr)
arr = np.where(np.logical_and(np.logical_and(np.logical_and(np.logical_and(drain<0,lons>282),lats<-72),lons<298),lats>-75),70,arr)

arr = np.where(np.logical_and(arr>65,cdist>250),-2,arr)

#arr = np.where(ds['arrival_point_id']==-1,-1,arr)

In [ ]:
#Overwrite masks and save as new file

ds['drainage_basin_id'][:] = drain
ds['calving_point_id'][:] = calv
ds['arrival_point_id'][:] = arr

ds.to_netcdf('../data/inputfiles/runoff_maps.nc')